In [ ]:
import pynq
import numpy as np
from tqdm import tqdm
import time
#from sklearn.metrics import average_precision_score, roc_auc_score

# 通过PYNQ将合成电路写入FPGA中
ol_loda = pynq.Overlay('./SCD_Inter_Full_16161616.bit')
mSCD = ol_loda.model_SCD_0
mM2M = ol_loda.Multi2Multi_0
mM2O = ol_loda.Multi2One_0

CONTROL_REGISTER = 0x0
mM2O.register_map

In [ ]:
def _load_via_txt(path):
    X = np.loadtxt(path, delimiter=",")
    return X

def get_data(data_file):
    #data_path = os.path.join(data_base_path, data_file)
    data_path = data_file
    if ".mat" in data_file:
        from scipy.io import loadmat
        f = loadmat(data_path)
        X = f['X']
    else:
        X = _load_via_txt(data_path)
    return X

X_train = get_data("./deepsig.dat")  # Load Aryhytmia data.
print("X_train.shape = ",X_train.shape)

In [ ]:
# 使用AXI DMA进行存储器传输
dma_i = ol_loda.axi_dma_0.sendchannel
dma_q = ol_loda.axi_dma_1.sendchannel
dma_v = ol_loda.axi_dma_2.recvchannel
dma_a = ol_loda.axi_dma_3.recvchannel
dma_f = ol_loda.axi_dma_4.recvchannel
# 处理大量数据
N = 2048 # X_train.shape[0]
Nout = 32768
NSamples = 100

buff_x = pynq.allocate(shape=(NSamples*N,), dtype=np.float32)
buff_y = pynq.allocate(shape=(NSamples*N,), dtype=np.float32)
buff_value = pynq.allocate(shape=(NSamples*Nout,), dtype=np.float32)
buff_al = pynq.allocate(shape=(NSamples*Nout,), dtype=np.int16)
buff_fl = pynq.allocate(shape=(NSamples*Nout,), dtype=np.int16)

# print("X_train[0:N] = ",X_train[0:N][:])


for j in range (NSamples):
    buff_x[j*N:(j+1)*N] = X_train[0:N,0]
    buff_y[j*N:(j+1)*N] = X_train[0:N,1]
    

In [ ]:
start = time.time()
mSCD.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2M.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2O.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

dma_i.transfer(buff_x)
dma_q.transfer(buff_y)
dma_v.transfer(buff_value)
dma_a.transfer(buff_al)
dma_f.transfer(buff_fl)
dma_i.wait()
dma_q.wait()
# dma_r.wait()
end = time.time()
mSCD.write(CONTROL_REGISTER,0x00) # stop
mM2M.write(CONTROL_REGISTER,0x00) # stop
mM2O.write(CONTROL_REGISTER,0x00) # stop
time_cost= end-start
print('=> Running on FPGA')
print(time_cost)
print()

In [ ]:
# print("buff_y[0:N] = ",buff_y[0:10])
# print("buff_y[-1] = ",buff_y[-1])
# del buff_x, buff_y

In [ ]:
buff_value[0]

In [ ]:
x = np.arange(0,809)
for i in x:
    print(i,":",buff_value[i],",",buff_al[i],",",buff_fl[i])
#     print(i,";",buff_z[i],",")

In [ ]:
import re
import math
dat_file = r"sparse_SCD_float_full16161616.dat"   
dm = []
al = []
fl = []
with open(dat_file, 'r') as file:
    text = file.read()
    data = re.split(',|\n', text)
    for i in range(int((len(data)-1)/3)):
        dm.append(float(data[i*3]))
        al.append(int(data[i*3+1]))
        fl.append(int(data[i*3+2]))

In [ ]:
rdm = np.zeros(len(dm))
ral = np.zeros(len(dm))
rfl = np.zeros(len(dm))

In [ ]:
for i in range(len(dm)):
    for j in range(len(np.where(al==buff_al[i])[-1])):
        if fl[np.where(al==buff_al[i])[0][j]] == buff_fl[i]:
#             print(i)
#             print(buff_al[i])
#             print(np.where(al==buff_al[i])[0][j])
#             print(fl[np.where(al==buff_al[i])[0][j]])
#             print(buff_fl[i])
#             print(buff_value[i])
#             print(dm[np.where(al==buff_al[i])[0][j]])
#             print('------------')
            rdm[i]=dm[np.where(al==buff_al[i])[0][j]]
            ral[i]=al[np.where(al==buff_al[i])[0][j]]
            rfl[i]=fl[np.where(al==buff_al[i])[0][j]]

In [ ]:
Psignal = np.sum(np.power(rdm,2))
# Pnoise  = np.var(rdm-buff_value[0:len(rdm)])
Pnoise  = np.sum(np.power(rdm-buff_value[0:len(rdm)],2))
print("The SQNR is: " , 10*math.log10(Psignal/Pnoise))

In [ ]:
Psignal = np.sum(np.power(rdm,2))
Pnoise  = np.var(rdm-buff_value[0:len(rdm)])
print("The SQNR is: " , 10*math.log10(Psignal/Pnoise))

In [ ]:
for i in range(len(dm)):
    print(str(i)+':'+str(rdm[i])+','+str(buff_value[i]))

In [ ]:
np.max(rdm-buff_value[0:len(rdm)])

In [ ]:
math.log2(np.max(rdm-buff_value[0:len(rdm)]))

In [ ]:
np.power([1,2,3],2)

In [ ]:
ral-buff_al[0:len(ral)]

In [ ]:
rfl-buff_fl[0:len(rfl)]